In [1]:
#imports

In [2]:
#function to create plotting environment?

## Experiment 1

Simulation:  
- visual score is linear with random intercept and slope per person  
- symptom: linear with fixed intercept and slope  
- error: None  
- npops=100  
- npersons=100  
    
Filter: None  
Sample: None  
Regression: Simple linear, Mixed effects with random intercept and slope  
Variations: More or less variance on the random intercept and slope (say: pmsigma=1, pmsigma=3)  

Purpose: Evaluate the bias under idealized conditions  
Hypothesis: There will be no bias at all  
Reasoning: The relationship is purely linear, no persons are discarded from the population, and each person will have at least two points to describe their straight line (in reality: they have one point per day)